In [ ]:
colab = False
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = '/content/drive/My Drive/Final_project/data'
    model_dir = '/content/drive/My Drive/Final_project/model'
    output_dir = '/content/drive/My Drive/Final_project/output'
else:
    data_dir = './data'
    model_dir = './model'
    output_dir = './output'




In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn
import numpy as np
import torchvision
import os
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from scipy.io import loadmat
from torch.autograd import Variable
import cv2
from sklearn.svm import LinearSVC
# from matplotlib import pyplot as plt
import torch.nn.functional as F
    
use_GPU = torch.cuda.is_available()
if use_GPU:
    print("Using CUDA")
    device = torch.device("cuda") 
else:
    print ("Using CPU")




In [ ]:
# Load data
from torch.utils.data.sampler import SubsetRandomSampler

transform = transforms.Compose([
                transforms.Resize([48, 48]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]) # orignally for ImageNet


svhn = {
    x: datasets.SVHN(root=data_dir, split= x, download=True, transform=transform)
    for x in ['train', 'test']
}

# use test dataset of cifar10 as negative control, and labeled as 10
cifar = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
cifar.targets[:] = [10]*len(cifar)

print (len(cifar))
print (len(svhn['train']))
print (len(svhn['test']))

comb_datasets = torch.utils.data.ConcatDataset([svhn['train'], cifar])

val_ratio = .01
random_seed= 42
np.random.seed(random_seed)


# Creating data indices for training and validation splits:
train_size = len(comb_datasets)
indices = list(range(train_size))

split = int(np.floor(val_ratio * train_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

dataloaders = {
    'train': torch.utils.data.DataLoader(
        comb_datasets, batch_size=128, shuffle=False, num_workers=8, sampler=train_sampler
    ),
    'val': torch.utils.data.DataLoader(
        comb_datasets, batch_size=128, shuffle=False, num_workers=8, sampler=val_sampler
    ),
    'test': torch.utils.data.DataLoader(
        svhn['test'], batch_size=128, shuffle=True, num_workers=8
    )
}


dataset_sizes = {'train': len(train_indices),
                 'val': len(val_indices),
                 'test': len(svhn['test'])}

for x in ['train', 'val', 'test']:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    

In [ ]:

train_pct = 1.0

def train(model, criterion, optimizer, scheduler, num_epochs=5):
    train_start = time.time()
    num_train_batches = len(dataloaders['train'])   
    num_val_batches = len(dataloaders['val'])
    num_test_batches = len(dataloaders['test'])
    
    best_acc = 0
    
    ave_loss_train = 0
    ave_loss_val = 0
    ave_loss_test = 0
    ave_acc_train = 0
    ave_acc_val = 0
    ave_acc_test = 0
    
    count = 0 # number of validation accuracy decreases
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        print ("Epoch {}/{}".format(epoch+1, num_epochs))
        print ('--' * 5)
        
        loss_train_sum = 0
        loss_val_sum = 0
        loss_test_sum = 0
        acc_train_sum = 0
        acc_val_sum = 0
        acc_test_sum = 0
        
        
        for i, data in enumerate(dataloaders['train']):
            
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, num_train_batches*train_pct), end='', flush=True)
                
            if i >= num_train_batches*train_pct:
                break
            
            images, labels = data
            
            images.requires_grad_(True)
            # labels.requires_grad_(True)
            
            if use_GPU:
                images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train_sum += loss.item()
            acc_train_sum += torch.sum(torch.eq(preds, labels.data))
            
            del images, labels, outputs, preds
            torch.cuda.empty_cache()
        
        # # save check point
        # model_name = 'VGG16'
        # date = 'Nov_25'
        # # status = 'pretrained'
        # status = 'random'
        # PATH = F'/content/drive/My Drive/Final_project/model/{model_name}_{status}_{date}_epoch_{epoch}.pt'
        # torch.save({
        #     'epoch': epoch,
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict(),
        #     'loss': loss,
        #     }, PATH)
        
        model.train(False)
        model.eval()
        
        print ()
        print ('Validating')

        for i, data in enumerate(dataloaders['val']):
            
            images, labels = data
            
            images.requires_grad_(True)
            # labels.requires_grad_(True)
            
            if use_GPU:
                images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val_sum += loss.item()
            acc_val_sum += torch.sum(torch.eq(preds, labels.data))
            
            del images, labels, outputs, preds
            torch.cuda.empty_cache()   
        
        
        
        print ()
        print ('testing')

        for i, data in enumerate(dataloaders['test']):
            
            images, labels = data
            
            images.requires_grad_(True)
            # labels.requires_grad_(True)
            
            if use_GPU:
                images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_test_sum += loss.item()
            acc_test_sum += torch.sum(torch.eq(preds, labels.data))
            
            del images, labels, outputs, preds
            torch.cuda.empty_cache() 
        
        avg_loss_train = loss_train_sum * 1.0/ dataset_sizes['train'] / train_pct
        avg_acc_train = acc_train_sum * 1.0/ dataset_sizes['train'] / train_pct
        avg_loss_val = loss_val_sum * 1.0 / dataset_sizes['val']
        avg_acc_val = acc_val_sum * 1.0/ dataset_sizes['val']
        avg_loss_test = loss_test_sum * 1.0/ dataset_sizes['test']
        avg_acc_test = acc_test_sum * 1.0 / dataset_sizes['test']
        
        epoch_length = time.time() - epoch_start
        
        print ()
        print ()
        print ("Epoch {} result: ".format(epoch+1))
        print ("Avg loss of training: {:.4f}".format(avg_loss_train))
        print ("Avg acc of training: {:.4f}".format(avg_acc_train))
        print ("Avg loss of validation: {:.4f}".format(avg_loss_val))
        print ("Avg acc of validation: {:.4f}".format(avg_acc_val))
        print ("Avg loss of test: {:.4f}".format(avg_loss_test))
        print ("Avg acc of test: {:.4f}".format(avg_acc_test))
        print ()
        print ("Epoch completed in {:.0f}m {:.0f}s".format(epoch_length // 60, epoch_length % 60))
        print ('--' * 5)
        print ()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())
            count = 0
        else:
            count +=1
        print ("count = ", count)
        print ("avg_acc_val", avg_acc_val)
        print ("best_acc", best_acc)

        # earlier stop
        if count >= 6:
            print ('earlier stop')
            break 
        
    training_time = time.time() - train_start
    print ()
    print ("Training completed in {:.0f}m {:.0f}s".format(training_time // 60, training_time % 60))
    print ("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model  




In [ ]:
import torch.nn.functional as F

drop_rate = 0.2

class my_model(nn.Module):
    
    def __init__(self):
        super(my_model, self).__init__()

        self._hidden1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=48),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(drop_rate)
        )
        self._hidden2 = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Dropout(drop_rate)
        )
        self._hidden3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(drop_rate)
        )
        self._hidden4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=160, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=160),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Dropout(drop_rate)
        )
        self._hidden5 = nn.Sequential(
            nn.Conv2d(in_channels=160, out_channels=192, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(drop_rate)
        )
        self._hidden6 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Dropout(0.2)
        )
        self._hidden7 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(drop_rate)
        )
        self._hidden8 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Dropout(drop_rate)
        )
        self._hidden9 = nn.Sequential(
            nn.Linear(192 * 3 * 3, 3072),
            nn.ReLU()
        )
        self._hidden10 = nn.Sequential(
            nn.Linear(3072, 3072),
            nn.ReLU()
        )

        self._output = nn.Sequential(nn.Linear(3072, 11))

    def forward(self, x):
        x = self._hidden1(x)
        x = self._hidden2(x)
        x = self._hidden3(x)
        x = self._hidden4(x)
        x = self._hidden5(x)
        x = self._hidden6(x)
        x = self._hidden7(x)
        x = self._hidden8(x)
        x = x.view(x.size(0), 192 * 3 * 3)
        x = self._hidden9(x)
        x = self._hidden10(x)

        output = self._output(x)  
        
        return output

# test = my_model()
# print(test)



In [ ]:
# run my model

my_model = my_model()

model_save_name = 'MyModel_SVHN_Dec_03.pt'

if colab = True:
    model_path = F'/content/drive/My Drive/Final_project/model/{model_save_name}' 
else:
    model_path = F'./model/{model_save_name}'
    

resume_training = False

if resume_training:
    my_model.load_state_dict(torch.load(model_path))
    print('My model loaded!')
    
if use_GPU:
    my_model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(my_model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

my_model = train(my_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)
torch.save(my_model.state_dict(), model_path)

